In [35]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Constantes e parâmetros iniciais
alpha = [1, -1, 0]
vetor_a = [1,1,3]
vetor_b = [
1 / 2 * (vetor_a[0] - vetor_a[1] - vetor_a[2]),
1 / 2 * (vetor_a[1] - vetor_a[2] - vetor_a[0]),
1 / 2 * (vetor_a[2] - vetor_a[0] - vetor_a[1]),
]
c = 3.86

# Parâmetros físicos
g = 10
g_0 = 8
kappa = 2.25 * 10**6
nu = kappa
kappa_0 = 1 / 48
nu_0 = kappa_0
l = 1000
vetor_h = [-1, 0, 0]
vetor_f = [0.1, 0, 0]

# Inicialização
r = 21
vetor_phi = [
math.cos(alpha[0] * r / l),
math.cos(alpha[1] * r / l),
math.cos(alpha[2] * r / l),
]

def calculate_UV(X, Y):
    U = []
    V = []
    for i in range(3):
        j = (i + 1) % 3
        k = (i + 2) % 3
        U.append(-vetor_b[j] * X[i] + c * Y[i])
        V.append(-vetor_b[k] * X[i] - c * Y[i])
    return U, V

def derivatives(X, Y, Z):
    U, V = calculate_UV(X, Y)
    dX = []
    dY = []
    dZ = []
    for i in range(3):
        j = (i + 1) % 3
        k = (i + 2) % 3
        dX.append(U[j] * U[k] + V[j] * V[k] - nu_0 * vetor_a[i] * X[i]  + Y[i] + vetor_a[i] * Z[i])
        dY.append(U[j] * Y[k] + Y[j] * V[k] - X[i]  - nu_0 * vetor_a[i] * Y[i])
        dZ.append(U[j] * (Z[k] - vetor_h[k])+ (Z[j] - vetor_h[j]) * V[k] - g_0 * X[i] - kappa_0 * vetor_a[i] * Z[i] + vetor_f[i])
    return dX, dY, dZ

def runge_kutta(dias, dt):
    t_max = dias*8
    n_steps = int(t_max / dt)  # 38400 iterações
    
    # Arrays para armazenar resultados
    t = np.linspace(0, t_max, n_steps)
    X = np.zeros((n_steps, 3))
    Y = np.zeros((n_steps, 3))
    Z = np.zeros((n_steps, 3))
    
    # Define condições iniciais
    X[0] = [0.10, 0.10, 0.10]
    Y[0] = [0.10, 0.10, 0.10]
    Z[0] = [0.10, 0.10, 0.10]
    
    for i in range(1, n_steps):
        # k1
        dx1, dy1, dz1 = derivatives(X[i-1], Y[i-1], Z[i-1])
        k1x = dt * np.array(dx1)  # Convertendo lista para array
        k1y = dt * np.array(dy1)
        k1z = dt * np.array(dz1)
        
        # k2
        dx2, dy2, dz2 = derivatives(
            X[i-1] + k1x/2,
            Y[i-1] + k1y/2,
            Z[i-1] + k1z/2
        )
        k2x = dt * np.array(dx2)
        k2y = dt * np.array(dy2)
        k2z = dt * np.array(dz2)
        
        # k3
        dx3, dy3, dz3 = derivatives(
            X[i-1] + k2x/2,
            Y[i-1] + k2y/2,
            Z[i-1] + k2z/2
        )
        k3x = dt * np.array(dx3)
        k3y = dt * np.array(dy3)
        k3z = dt * np.array(dz3)
        
        # k4
        dx4, dy4, dz4 = derivatives(
            X[i-1] + k3x,
            Y[i-1] + k3y,
            Z[i-1] + k3z
        )
        k4x = dt * np.array(dx4)
        k4y = dt * np.array(dy4)
        k4z = dt * np.array(dz4)
        
        # Atualização
        X[i] = X[i-1] + (k1x + 2*k2x + 2*k3x + k4x)/6
        Y[i] = Y[i-1] + (k1y + 2*k2y + 2*k3y + k4y)/6
        Z[i] = Z[i-1] + (k1z + 2*k2z + 2*k3z + k4z)/6
    
    return t, X, Y, Z

# Execute a simulação para obter os dados
dias = 1
dt = 1/96
t, X, Y, Z = runge_kutta(dias, dt)

# Crie a figura
fig, ax = plt.subplots(figsize=(12, 8))

# Plotar os 400 dias completos
ax.plot(t, X[:, 0], '-', color='red', label='x₁', linewidth=1)
ax.plot(t, Y[:, 0], '-', color='blue', label='y₁', linewidth=1)
ax.plot(t, Z[:, 0], '-', color='green', label='z₁', linewidth=1)

# Configurar os limites dos eixos
ax.set_xlim(0, dias*8)
ax.set_ylim(-0.4, 0.6)

# Configurar os rótulos dos eixos
ax.set_xlabel('days')
ax.set_ylabel('')

# Adicionar a grade
ax.grid(True, linestyle='-', alpha=1)

# Adicionar legenda
ax.legend()

plt.tight_layout()
plt.show()

"\nplt.figure(figsize=(10, 6))\nplt.plot(t[:96], X[:96, 0], '--', label='x₁')\nplt.plot(t[:96], Y[:96, 0], '-', linewidth=2, label='y₁')\nplt.plot(t[:96], Z[:96, 0], '-', label='z₁')\nplt.xlabel('days')\nplt.ylim(-0.4, 0.6)\nplt.grid(True)\nplt.legend()\nplt.title('First day variations')\nplt.show()\n"